# **Introducción a la IA** — Hoja de **Ejemplos** (Imágenes)
**Máster: IA y Fabricación Aditiva**  
**Lectura 1** — *Procesado de imagen digital*  
**Fecha:** 2025-10-06

> Este cuaderno está diseñado para **Google Colab** o local.  
> Cambia **1–2 parámetros** en cada bloque ***TRY IT*** y vuelve a ejecutar.


---
## 🎯 Objetivos
- Entender la representación numérica de imágenes (matrices/tensores).
- Manipular regiones (*slicing*), inversiones y trasposición.
- Operar con canales RGB, normalización y formatos HWC/CHW.
- Implementar reescalado (vecino más cercano) y un filtro de media.
- Crear pequeñas modificaciones y observar su efecto inmediato.


In [ ]:

# === Preparación del entorno ===
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 120

def show(img, title=None):
    plt.figure(figsize=(5,3))
    if img.ndim == 2:
        plt.imshow(img, cmap='gray', vmin=img.min(), vmax=img.max())
    else:
        plt.imshow(img)
    if title: plt.title(title)
    plt.axis('off'); plt.show()

print("Entorno listo ✔")


---
## 0) Imagen base (sintética)
**TRY IT:** modifica tamaño, radio y degradados. Reejecuta la celda.


In [ ]:

# === PARÁMETROS (modifica) ===
H, W = 240, 360
circle_radius = 60
gradient_R = True    # degradado horizontal
gradient_G = True    # degradado vertical
gradient_B = False   # sin degradado en B
# === FIN PARÁMETROS ===

x = np.linspace(0, 1, W)
y = np.linspace(0, 1, H)
X, Y = np.meshgrid(x, y)

R = (X * 255).astype(np.uint8) if gradient_R else np.zeros((H,W), np.uint8)
G = (Y * 255).astype(np.uint8) if gradient_G else np.zeros((H,W), np.uint8)
B = ((1-X) * 255).astype(np.uint8) if gradient_B else np.zeros((H,W), np.uint8)

YY, XX = np.ogrid[:H, :W]
mask = (XX - W//2)**2 + (YY - H//2)**2 <= circle_radius**2
for C in (R, G, B):
    C[mask] = 255

img = np.stack([R, G, B], axis=2)  # HWC
show(img, "Imagen base sintética (HWC)")
print("img.shape:", img.shape, "| dtype:", img.dtype)


---
## 1) *Slicing* (recortes)
**TRY IT:** cambia `y0,y1,x0,x1` y observa el recorte.


In [ ]:

# === PARÁMETROS (modifica) ===
y0, y1 = 60, 200
x0, x1 = 100, 300
# === FIN ===

crop = img[y0:y1, x0:x1, :]
show(crop, f"Recorte: y[{y0}:{y1}] x[{x0}:{x1}]")


---
## 2) Inversiones
**TRY IT:** activa/desactiva `flip_vertical` y `flip_horizontal`.


In [ ]:

# === PARÁMETROS (modifica) ===
flip_vertical = True
flip_horizontal = False
# === FIN ===

out = img.copy()
if flip_vertical:   out = out[::-1, :, :]
if flip_horizontal: out = out[:, ::-1, :]
show(out, f"Inversiones - vertical={flip_vertical}, horizontal={flip_horizontal}")


---
## 3) Trasposición de ejes
**TRY IT:** cambia `axes` (por ejemplo `(1,0,2)`).


In [ ]:

# === PARÁMETROS (modifica) ===
axes = (1, 0, 2)  # (H,W,C) -> (W,H,C)
# === FIN ===

img_T = np.transpose(img, axes)
show(img_T, f"Trasposición axes={axes}")
print("shape antes:", img.shape, " | shape después:", img_T.shape)


---
## 4) Canales RGB
**TRY IT:** activa `show_R/G/B` para visualizar por canal.


In [ ]:

# === PARÁMETROS (modifica) ===
show_R, show_G, show_B = True, True, True
# === FIN ===

if show_R: show(img[...,0], "Canal R")
if show_G: show(img[...,1], "Canal G")
if show_B: show(img[...,2], "Canal B")


---
## 5) Composición por canales
**TRY IT:** asigna canales a mantener (e.g. `"RB"`, `"G"`, `"RGB"`).


In [ ]:

# === PARÁMETROS (modifica) ===
keep = "R"   # opciones: "R","G","B","RG","RB","GB","RGB"
# === FIN ===

out = np.zeros_like(img)
if "R" in keep: out[...,0] = img[...,0]
if "G" in keep: out[...,1] = img[...,1]
if "B" in keep: out[...,2] = img[...,2]
show(out, f"Canales activos: {keep}")


---
## 6) Normalización [0,1] y factor `alpha`
**TRY IT:** cambia `alpha` y observa el efecto.


In [ ]:

# === PARÁMETROS (modifica) ===
alpha = 1.0
# === FIN ===

img_f = img.astype(np.float32) / 255.0
img_f = np.clip(img_f * alpha, 0.0, 1.0)
show(img_f, f"Normalizada (alpha={alpha})")
print(img_f.dtype, img_f.min(), img_f.max())


---
## 7) HWC ↔ CHW e índice lineal
**TRY IT:** cambia `(x,y,c)` y comprueba el índice lineal.


In [ ]:

# === PARÁMETROS (modifica) ===
x, y, c = 15, 192, 2  # c=0(R),1(G),2(B)
# === FIN ===

img_chw = np.transpose(img, (2,0,1))  # CHW
H_, W_ = img.shape[0], img.shape[1]
index = x + y*W_ + c*W_*H_
print("HWC:", img.shape, " | CHW:", img_chw.shape)
print(f"Índice lineal para (x={x}, y={y}, c={c}) =", index)


---
## 8) Reescalado (vecino más cercano)
**TRY IT:** cambia `scale` (2, 3, ...).


In [ ]:

# === PARÁMETROS (modifica) ===
scale = 2
# === FIN ===

H2, W2 = img.shape[0]*scale, img.shape[1]*scale
up = np.zeros((H2, W2, 3), dtype=img.dtype)
for yy in range(H2):
    for xx in range(W2):
        y_src = yy // scale
        x_src = xx // scale
        up[yy, xx] = img[y_src, x_src]
show(up, f"Reescalado NN (scale={scale}x)")


---
## 9) Filtro de media k×k (en un canal)
**TRY IT:** cambia `channel` y `kernel_size` (impar).


In [ ]:

# === PARÁMETROS (modifica) ===
channel = 0          # 0=R,1=G,2=B
kernel_size = 3      # 3,5,7...
# === FIN ===

k = kernel_size
assert k % 2 == 1, "Usa tamaño impar (3,5,7,...)"
pad = k // 2

C = img[...,channel].astype(np.float32)
Hk, Wk = C.shape
out = np.zeros_like(C)
for i in range(pad, Hk-pad):
    for j in range(pad, Wk-pad):
        patch = C[i-pad:i+pad+1, j-pad:j+pad+1]
        out[i,j] = patch.mean()
blur = img.copy()
blur[...,channel] = np.clip(out, 0, 255).astype(np.uint8)
show(blur, f"Media {k}×{k} en canal {channel}")


---
## 10) Grises con pesos
**TRY IT:** modifica `wR,wG,wB`.


In [ ]:

# === PARÁMETROS (modifica) ===
wR, wG, wB = 1/3, 1/3, 1/3    # prueba 0.299, 0.587, 0.114
# === FIN ===

g = (wR*img[...,0] + wG*img[...,1] + wB*img[...,2]).astype(np.float32)
g = g / max(1e-6, (wR+wG+wB))
g = np.clip(g, 0, 255).astype(np.uint8)
show(g, f"Grises (wR={wR}, wG={wG}, wB={wB})")


---
## 11) Rejilla y píxel marcado
**TRY IT:** ajusta `step` y coord. `(px,py)`.


In [ ]:

# === PARÁMETROS (modifica) ===
draw_grid = True; step = 30
px, py = 15, 40               # (x,y)
color = (255, 0, 0)           # rojo
# === FIN ===

overlay = img.copy()
if draw_grid:
    overlay[::step, :, :] = 0
    overlay[:, ::step, :] = 0

mark = overlay.copy()
if 0 <= py < mark.shape[0] and 0 <= px < mark.shape[1]:
    mark[py, px] = color
show(mark, f"Grid step={step}, pixel ({px},{py})")


---
## 12) Guardar resultado a disco
**TRY IT:** cambia `filename` y el objeto a guardar.


In [ ]:

# === PARÁMETROS (modifica) ===
filename = "resultado.png"
image_to_save = img   # prueba con 'g', 'blur', 'mark', etc.
# === FIN ===

import matplotlib.pyplot as plt
plt.imsave(filename, image_to_save)
print("Guardado:", filename)
